In [1]:
import pandas as pd
import psycopg2
import requests

#### postgresql connection callable

# connection = psycopg2.connect(
#    host="localhost",
#    user="postgres",
#    password="vijay45",
#    database="postgres"

# )

# cursor=connection.cursor()

{ *** 1.  COLLECT THE COMPETITION DATA FROM THE API ENDPOINTS *** }

In [2]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=3cx83hgyhyf1YOaeIdPIxYu10bw5H0JSANgpnVSI"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

competition_data =response['competitions']

In [11]:
# FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR CATEGORY_DF AND Category_DF

category_id = []
category_name = []
for value in competition_data:
    category_id.append(value['category']['id'])
    category_name.append(value['category']['name'])

data = {'category_id': category_id, 'category_name': category_name}
category_df = pd.DataFrame(data)

print(category_df.duplicated().sum())

category_df = category_df.drop_duplicates()

category_df.duplicated().sum()

# FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR CATEGORY_DF AND COMPETITONS_DF

competition_id = []
competition_name = []
parent_id = []
competition_type = []
gender = []
category_id = []

for value in competition_data:
    competition_id.append(value.get('id'))
    competition_name.append(value.get('name'))
    parent_id.append(value.get('parent_id'))
    competition_type.append(value.get('type'))
    gender.append(value.get('gender'))
    category_id.append(value['category']['id'])

data = {'competition_id': competition_id, 'competition_name': competition_name, 'parent_id': parent_id, 'type': competition_type, 'gender': gender, 'category_id': category_id}
competitions_df = pd.DataFrame(data)

competitions_df.drop_duplicates()

5872


,competition_id,competition_name,parent_id,type,gender,category_id
0,sr:competition:620,Hopman Cup,None,mixed,mixed,sr:category:181
1,sr:competition:660,World Team Cup,None,mixed,men,sr:category:3
2,sr:competition:990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,sr:competition:1207,Championship International Series,None,singles,women,sr:category:6
4,sr:competition:2100,Davis Cup,None,mixed,men,sr:category:76
...,...,...,...,...,...,...
5885,sr:competition:45965,UTR Kawaguchi W01,None,singles,women,sr:category:2517
5886,sr:competition:45967,UTR Olomouc W02,None,singles,women,sr:category:2517
5887,sr:competition:45969,UTR Tigre W02,None,singles,women,sr:category:2517
5888,sr:competition:45971,UTR Boca Raton W03,sr:competition:45363,singles,women,sr:category:2517


In [13]:

# Establish the database connection

connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="vijay45",
    database="postgres"
)
cursor = connection.cursor()

# CREATING SQL TABLE FOR [CATEGORYTABLE]

cursor.execute("""
    CREATE TABLE IF NOT EXISTS categorytable (
        category_id VARCHAR(50) PRIMARY KEY,
        category_name VARCHAR(100) NOT NULL
    )
""")

# CREATING SQL TABLE FOR [COMPETITONTABLE]

cursor.execute("""
    CREATE TABLE IF NOT EXISTS competitiontable (
        competition_id VARCHAR(50) PRIMARY KEY,
        competition_name VARCHAR(100) NOT NULL,
        parent_id VARCHAR(50) NULL,
        type VARCHAR(20) NOT NULL,
        gender VARCHAR(10) NOT NULL,
        category_id VARCHAR(50) REFERENCES categories(category_id)
    )
""")

 
# inserting the data into postgreSQL

for index, row in category_df.iterrows():
    cursor.execute("""
    INSERT INTO categorytable(category_id, category_name)
    VALUES (%s, %s);
    """, (row['category_id'], row['category_name']))

for index, row in competitions_df.iterrows():
    cursor.execute("""
    INSERT INTO competitiontable (competition_id, competition_name, parent_id, type, gender, category_id)
    VALUES (%s, %s, %s, %s, %s, %s);
    """, (row['competition_id'], row['competition_name'], row['parent_id'], row['type'], row['gender'], row['category_id']))


connection.commit()
connection.close()

print("Data has been inserted into postgresql!")

Data has been inserted into postgresql!
